In [5]:

def generate_protocols(input_csv, protocol_group, random_class_order, disconnect_device_on_finish, sensor_config):
    # Read the CSV file and fill NaN values with an empty string
    df = pd.read_csv(input_csv).fillna("")
    current_json_data = None
    current_protocol_name = ""

    for index, row in df.iterrows():
        class_name = row['class_name']
        if class_name.lower() == 'protocol':
            # If there is an existing protocol, save it before starting a new one
            if current_json_data is not None:
                filename = f"{current_protocol_name}.json"
                with open(filename, 'w') as json_file:
                    json.dump(current_json_data, json_file, indent=2)
                print(f"Generated protocol file: {filename}")

            # Start a new protocol
            current_protocol_name = row['activeClass_name']
            current_json_data = {
                "name": current_protocol_name,
                "parameters": [],
                "active": True,
                "classes": [],
                "randomClassOrder": random_class_order,
                "preamble": row['preamble'] if pd.notna(row['preamble']) else "",
                "epilogue": row['epilogue'] if pd.notna(row['epilogue']) else "",
                "group": protocol_group,
                "disconnectDeviceOnFinish": disconnect_device_on_finish,
                "expectedDeviceTypes": [  # Add expectedDeviceTypes here
                    {
                        "sensorConfigurations": sensor_config
                    }
                ]
            }
        else:
            # Process non-protocol rows
            if current_json_data is not None:
                class_structure = {
                    "class": class_name,
                    "intervals": [],
                    "repetitions": int(row['repetitions']) if pd.notna(row['repetitions']) else 1,
                    "preamble": row['preamble'] if pd.notna(row['preamble']) else ""
                }
                # Add intervals with bookending inactive durations
                inactive_duration = int(row['inactive_duration']) if pd.notna(row['inactive_duration']) else 0
                active_duration = int(row['active_duration']) if pd.notna(row['active_duration']) else 0
                if inactive_duration > 0:
                    # Add an inactive interval before the active one
                    class_structure['intervals'].append({"label": "inactive", "durationInMillis": inactive_duration})
                if active_duration > 0:
                    # Add the active interval
                    class_structure['intervals'].append({"label": row['activeClass_name'], "durationInMillis": active_duration})
                if inactive_duration > 0:
                    # Add another inactive interval after the active one
                    class_structure['intervals'].append({"label": "inactive", "durationInMillis": inactive_duration})
                # Append class structure to the current protocol
                current_json_data["classes"].append(class_structure)

    # Save the last protocol if it exists
    if current_json_data is not None:
        filename = f"{current_protocol_name}.json"
        with open(filename, 'w') as json_file:
            json.dump(current_json_data, json_file, indent=2)
        print(f"Generated protocol file: {filename}")


In [6]:
randomClassOrder = False
disconnectDeviceOnFinish = True
example_csv = "protocol_example.csv"
sensor_config = [
    {
        "sensorType": "SENSOR_TYPE_EMG",
        "samplingRate": 800
    },
    {
        "sensorType": "SENSOR_TYPE_GYROSCOPE",
        "samplingRate": 4
    },
    {
        "sensorType": "SENSOR_TYPE_ACCELEROMETER",
        "samplingRate": 400,
        "brief": "16g"
    }#,
   # {
   #     "sensorType": "SENSOR_TYPE_PPG",
   #     "samplingRate": 30
   # },
   # {
   #     "sensorType": "SENSOR_TYPE_EDA",
   #     "samplingRate": 30
   # }
]

generate_protocols(
    input_csv=example_csv, 
    protocol_group="test", 
    random_class_order=randomClassOrder, 
    disconnect_device_on_finish=disconnectDeviceOnFinish, 
    sensor_config=sensor_config
)

Generated protocol file: test1.json
Generated protocol file: test2.json
